<a href="https://colab.research.google.com/github/graylan0/GPTVision-plugin/blob/main/LLama_Vision_Broken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin

--2023-11-17 12:02:41--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 65.8.178.12, 65.8.178.27, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/3bfdde943555c78294626a6ccd40184162d066d39774bd2c98dae24943d32cc3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1700481761&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDQ4MTc2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1MmM4ZDBkYWF

In [3]:
!pip install requests
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00


In [4]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822213 sha256=e50dcc5ba0a987be15b80c816e1a4692c615c33e991e47700d55d7f336a77628
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [1]:
import json
import base64
import requests
from PIL import Image
from io import BytesIO
from llama_cpp import Llama
import logging
from transformers import CLIPProcessor, CLIPModel

logging.basicConfig(level=logging.DEBUG)

llm = Llama(model_path="llama-2-7b-chat.ggmlv3.q8_0.bin", n_gpu_layers=-1, n_ctx=3900)
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def encode_image_to_base64(image_data):
    image = Image.open(BytesIO(image_data))
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    image.thumbnail((512, 512), image.thumbnail((512, 512), Image.Resampling.LANCZOS))
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    return base64.b64encode(buffer.getvalue()).decode('utf-8')

def resize_image(image_data):
    try:
        image = Image.open(BytesIO(image_data))
        if image.mode == 'RGBA':
            image = image.convert('RGB')
        image.thumbnail((512, 512), Image.ANTIALIAS)
        buffer = BytesIO()
        image.save(buffer, format="JPEG")
        return buffer.getvalue()
    except Exception as e:
        logging.error(f"Error resizing image: {e}")
        return None

def multi2vec_process_image(image_url, weaviate_instance_url, class_name):
    try:
        response = requests.get(image_url)
        if response.status_code != 200:
            logging.error(f"Failed to download image: HTTP {response.status_code}")
            return None
        image_data = response.content
        resized_image_data = resize_image(image_data)
        if resized_image_data is None:
            return None
        base64_encoded_image = encode_image_to_base64(resized_image_data)
        data_object = {"class": class_name, "properties": {"image": base64_encoded_image}}
        response = requests.post(f"{weaviate_instance_url}/objects", json=data_object)
        if response.status_code == 200:
            return response.json()
        else:
            logging.error(f"Failed to vectorize image: HTTP {response.status_code}")
            return None
    except Exception as e:
        logging.error(f"Error in multi2vec_process_image: {e}")
        return None

def get_clip_summary(image_data):
    try:
        # Load the image
        image = Image.open(BytesIO(image_data))

        # Prepare the inputs for the CLIP model
        # Note: We're using a dummy text input here because CLIP expects both text and image inputs.
        # You can replace "a photo" with a more suitable placeholder text if needed.
        inputs = clip_processor(text=["a photo"], images=image, return_tensors="pt", padding=True)

        # Get the outputs from the CLIP model
        outputs = clip_model(**inputs)

        # Process the outputs
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)

        # Return the most probable label (as an example)
        return probs.argmax().item()
    except Exception as e:
        logging.error(f"Error in get_clip_summary: {e}")
        return None
def calculate_token_usage(prompt, vectorized_data, max_tokens=3999):
    prompt_tokens = len(prompt.split())
    available_tokens = max_tokens - prompt_tokens
    tokens_per_number = 1
    max_data_length = available_tokens // tokens_per_number
    return max_data_length

def intelligent_chunking(vectorized_data, max_chunk_size):
    return [vectorized_data[i:i + max_chunk_size] for i in range(0, len(vectorized_data), max_chunk_size)]

def llama2_process_vectorized_image(vectorized_image, custom_prompt):
    max_data_length = calculate_token_usage(custom_prompt, vectorized_image['vector'])
    chunks = intelligent_chunking(vectorized_image['vector'], max_data_length)
    processed_outputs = []
    for index, chunk in enumerate(chunks):
        rules_based_prompt = (
            f"<sys>You are inspecting an Image please describe it.{index+1}</sys> "
            f"[INST]Adhere to the following Image Inspection AI Vector Space rules: "
            "1. Analyze the data for patterns. "
            "2. Identify key features and characteristics. "
            "3. Provide a descriptive analysis of the image content. "
            "4. Ensure accuracy and clarity in the description. "
            f"Chunk Data: {json.dumps(chunk)}[/INST]"
        )
        combined_prompt = f"{custom_prompt}. {rules_based_prompt}"
        logging.debug(f"Prompt for Llama model: {combined_prompt}")
        try:
            output = llm(combined_prompt, max_tokens=300, stop=["\n"], echo=True)
            if 'text' in output:
                processed_outputs.append(output['text'])
            else:
                logging.error(f"No text found in output for chunk {index+1}")
        except Exception as e:
            logging.error(f"Error processing chunk {index+1} with Llama model: {e}")
    return ' '.join(processed_outputs)

weaviate_instance_url = "https://roughly-sure-whale.ngrok-free.app/v1"
class_name = "BindExample"
image_url = "https://user-images.githubusercontent.com/34530588/258687329-1a9f08a3-f092-4180-a854-4ab48a04cf99.png"

response = requests.get(image_url)
if response.status_code == 200:
    image_data = response.content
    clip_summary = get_clip_summary(image_data)
    vectorized_image = multi2vec_process_image(image_url, weaviate_instance_url, class_name)
    if vectorized_image:
        vectorized_image['clip_summary'] = clip_summary
        custom_prompt = "Describe the content of this image"
        processed_data = llama2_process_vectorized_image(vectorized_image, custom_prompt)
        if processed_data:
            print(processed_data)
        else:
            print("Failed to process image data.")
    else:
        print("Failed to vectorize image.")
else:
    logging.error(f"Failed to download image: HTTP {response.status_code}")
    print("Failed to retrieve image data.")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 
<ipython-input-1-610f349b8b2b>:30: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image.thumbnail((512, 512), Image.ANTIALIAS)
ERROR:root:No text found in output for chunk 1


Failed to process image data.
